In [1]:
import pandas as pd
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch
from tqdm import tqdm

# Load the parquet file
df = pd.read_parquet("data/bluesky_text_embeddings (1).parquet")

# Unpack the binary embeddings
def unpack_embeddings(packed_bytes):
    return np.unpackbits(np.frombuffer(packed_bytes, dtype=np.uint8))

# Apply unpacking to get original binary embeddings
df['embeddings'] = df['embeddings'].apply(unpack_embeddings)

# Now you can look at the first few rows to verify
print("First embedding shape:", len(df['embeddings'].iloc[0]))
print(df[['item_id', 'embeddings']].head())

First embedding shape: 128
   item_id                                         embeddings
0  3460233  [1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, ...
1  3044498  [1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, ...
2  1582998  [1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, ...
3  5436174  [1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, ...
4  1582999  [1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, ...


In [2]:
import pandas as pd
import numpy as np
import duckdb

# 1. Load and unpack embeddings
con = duckdb.connect()
con.execute("""
    CREATE TABLE embeddings AS SELECT * FROM read_parquet('data/bluesky_text_embeddings (1).parquet');
""")
post_embeddings_df = con.execute("SELECT * FROM embeddings").fetchdf()

def unpack_embeddings(packed_bytes):
    return np.unpackbits(np.frombuffer(packed_bytes, dtype=np.uint8))

post_embeddings_df['embeddings'] = post_embeddings_df['embeddings'].apply(unpack_embeddings)

# 2. Load interactions
con.execute("""
    CREATE TABLE interactions AS SELECT * FROM read_csv('data/bluesky.csv');
""")
interactions_df = con.execute("SELECT * FROM interactions").fetchdf()

# 3. Join interactions with post embeddings
joined_df = interactions_df.merge(
    post_embeddings_df,
    left_on='destination_node',
    right_on='item_id',
    how='inner'
)

# 4. Group by user and create user embeddings
user_embeddings = joined_df.groupby('source_node')['embeddings'].agg(
    lambda x: np.mean(list(x), axis=0)
).reset_index()
user_embeddings.columns = ['user_id', 'user_embedding']

# 5. Create final DataFrame with all information
final_df = joined_df.merge(
    user_embeddings,
    left_on='source_node',
    right_on='user_id',
    how='inner'
)

# Verify the data
print("Final DataFrame shape:", final_df.shape)
print("\nColumns:", final_df.columns.tolist())
print("\nSample user-post pair:")
sample = final_df.iloc[0]
print(f"User ID: {sample['source_node']}")
print(f"Post ID: {sample['destination_node']}")
print(f"User embedding (first 10):", sample['user_embedding'][:10])
print(f"Post embedding (first 10):", sample['embeddings'][:10])

Final DataFrame shape: (16943200, 8)

Columns: ['source_node', 'destination_node', 'timestamp', 'edge_label', 'item_id', 'embeddings', 'user_id', 'user_embedding']

Sample user-post pair:
User ID: 50947
Post ID: 3460233
User embedding (first 10): [0.96829477 0.         0.86032562 0.48671808 0.95886889 0.04798629
 0.15595544 0.9991431  0.46615253 0.48500428]
Post embedding (first 10): [1 0 1 1 1 0 0 1 1 0]


In [3]:
# Pick a random user_id to examine
user_id = final_df['user_id'].iloc[21]

# Get their embedding
user_emb = final_df[final_df['user_id'] == user_id]['user_embedding'].iloc[0]

# Get all posts this user liked
liked_posts = final_df[final_df['user_id'] == user_id]['destination_node'].values

# Get embeddings for these posts
liked_post_embeddings = post_embeddings_df[post_embeddings_df['item_id'].isin(liked_posts)]['embeddings'].values

print(f"User {user_id}:")
print(f"Number of liked posts: {len(liked_posts)}")
print(f"\nUser embedding (first 20 values):\n{user_emb[:20]}")
print(f"\nLiked posts embeddings (first 3 posts, first 20 values):")
for i, emb in enumerate(liked_post_embeddings[:9]):
    print(f"Post {i}: {emb[:20]}")

# Verify that user embedding is indeed the average
avg_liked_embeddings = np.mean(liked_post_embeddings, axis=0)
print(f"\nVerification - are user embeddings the average of liked posts?")
print(f"Max difference: {np.max(np.abs(user_emb - avg_liked_embeddings))}")

User 35444:
Number of liked posts: 299

User embedding (first 20 values):
[0.95317726 0.         0.85284281 0.56856187 0.9632107  0.05685619
 0.13043478 0.98996656 0.4548495  0.55518395 0.34448161 1.
 0.33110368 0.11371237 0.05016722 0.33779264 0.03344482 0.94983278
 0.55518395 0.5819398 ]

Liked posts embeddings (first 3 posts, first 20 values):
Post 0: [1 0 0 1 1 0 0 1 1 0 0 1 0 0 0 0 0 1 1 0]
Post 1: [1 0 0 1 1 0 0 1 0 0 1 1 1 0 0 0 0 1 1 1]
Post 2: [1 0 0 1 1 0 0 1 1 1 0 1 0 0 0 1 0 1 1 1]
Post 3: [1 0 1 1 1 0 0 1 0 0 0 1 0 1 0 0 0 1 0 1]
Post 4: [1 0 1 1 1 0 0 1 0 0 0 1 0 0 0 1 0 1 0 0]
Post 5: [1 0 1 0 0 0 0 1 1 1 0 1 1 1 0 0 0 1 1 0]
Post 6: [1 0 1 0 1 0 0 1 0 0 1 1 0 0 0 0 0 1 1 0]
Post 7: [1 0 1 1 1 0 0 1 0 0 1 1 0 0 0 1 0 1 0 1]
Post 8: [1 0 1 1 1 0 0 1 0 1 0 1 1 0 0 0 0 1 1 0]

Verification - are user embeddings the average of liked posts?
Max difference: 1.0


In [4]:
import torch
from torch.utils.data import Dataset, DataLoader
from model import UserTower, PostTower, TwoTowerModel
from sklearn.model_selection import train_test_split
import torch.nn as nn

class UserPostDataset(Dataset):
    def __init__(self, df, negative_samples=1):
        self.df = df
        self.negative_samples = negative_samples
        # Precompute user's positive posts for faster lookup
        self.user_positives = {
            user: set(group['destination_node'].values) 
            for user, group in df.groupby('source_node')
        }
        self.all_posts = df['destination_node'].unique()
        # Calculate total length including negative samples
        self.length = len(df) * (self.negative_samples + 1)
        
    def __len__(self):
        return self.length  # Return integer length
    
    def __getitem__(self, idx):
        interaction_idx = idx // (self.negative_samples + 1)
        is_positive = idx % (self.negative_samples + 1) == 0
        
        row = self.df.iloc[interaction_idx]
        user_id = row['source_node']
        
        if is_positive:
            post_emb = row['embeddings']
            user_emb = row['user_embedding']
        else:
            # Simple random sampling without checking
            neg_post_idx = np.random.choice(len(self.df))
            neg_post = self.df.iloc[neg_post_idx]
            post_emb = neg_post['embeddings']
            user_emb = row['user_embedding']
        
        return (
            torch.tensor(user_emb, dtype=torch.float32),
            torch.tensor(post_emb, dtype=torch.float32),
            torch.tensor(1.0 if is_positive else 0.0, dtype=torch.float32)
        )

# Create datasets and dataloaders
train_df, val_df = train_test_split(final_df, test_size=0.2, random_state=42)

train_dataset = UserPostDataset(train_df)
val_dataset = UserPostDataset(val_df)

train_loader = DataLoader(train_dataset, batch_size=1024, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=1024, shuffle=False, num_workers=4)

In [5]:
# # 2. Split data and create dataloaders
# train_df, val_df = train_test_split(final_df, test_size=0.2, random_state=42)
# all_post_ids = final_df['destination_node'].unique()

# train_dataset = UserPostDataset(train_df, all_post_ids)
# val_dataset = UserPostDataset(val_df, all_post_ids)

# train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=4)
# val_loader = DataLoader(val_dataset, batch_size=128, shuffle=False, num_workers=4)

# # 3. Initialize model
# embedding_dim = len(final_df['embeddings'].iloc[0])  # Should be 128
# hidden_dims = [64, 32]

# user_tower = UserTower(embedding_dim=embedding_dim, hidden_dims=hidden_dims)
# post_tower = PostTower(embedding_dim=embedding_dim, hidden_dims=hidden_dims)
# model = TwoTowerModel(user_tower, post_tower)

# # 4. Training setup
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# print(f"Using device: {device}")

# model = model.to(device)
# optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
# criterion = nn.BCEWithLogitsLoss()

# 5. Training loop
# num_epochs = 3
# best_val_loss = float('inf')

# for epoch in range(num_epochs):
#     # Training phase
#     model.train()
#     train_loss = 0
#     for batch_idx, (user_features, post_features, labels) in enumerate(train_loader):
#         user_features = user_features.to(device)
#         post_features = post_features.to(device)
#         labels = labels.to(device)
        
#         optimizer.zero_grad()
#         user_emb, post_emb = model(user_features, post_features)
        
#         # Compute similarity scores
#         scores = torch.sum(user_emb * post_emb, dim=1)
#         loss = criterion(scores, labels)
        
#         loss.backward()
#         optimizer.step()
        
#         train_loss += loss.item()
        
#         if batch_idx % 100 == 0:
#             print(f'Epoch {epoch}, Batch {batch_idx}, Loss: {loss.item():.4f}')
    
#     avg_train_loss = train_loss / len(train_loader)
    
#     # Validation phase
#     model.eval()
#     val_loss = 0
#     correct_predictions = 0
#     total_predictions = 0
    
#     with torch.no_grad():
#         for user_features, post_features, labels in val_loader:
#             user_features = user_features.to(device)
#             post_features = post_features.to(device)
#             labels = labels.to(device)
            
#             user_emb, post_emb = model(user_features, post_features)
#             scores = torch.sum(user_emb * post_emb, dim=1)
#             loss = criterion(scores, labels)
            
#             val_loss += loss.item()
            
#             # Calculate accuracy
#             predictions = (torch.sigmoid(scores) > 0.5).float()
#             correct_predictions += (predictions == labels).sum().item()
#             total_predictions += labels.size(0)
    
#     avg_val_loss = val_loss / len(val_loader)
#     accuracy = correct_predictions / total_predictions
    
#     print(f'Epoch {epoch}:')
#     print(f'  Training Loss: {avg_train_loss:.4f}')
#     print(f'  Validation Loss: {avg_val_loss:.4f}')
#     print(f'  Validation Accuracy: {accuracy:.4f}')
    
#     # Save best model
#     if avg_val_loss < best_val_loss:
#         best_val_loss = avg_val_loss
#         torch.save(model.state_dict(), 'best_model.pt')

In [6]:
embedding_dim=128

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [7]:
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm

# Modified model with simpler architecture for testing
class SimpleTwoTowerModel(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.user_tower = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
        )
        self.post_tower = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
        )
        
    def forward(self, user_features, post_features):
        user_emb = F.normalize(self.user_tower(user_features), p=2, dim=1)
        post_emb = F.normalize(self.post_tower(post_features), p=2, dim=1)
        return user_emb, post_emb

# Try training with this simpler model
model = SimpleTwoTowerModel(embedding_dim).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.BCEWithLogitsLoss()

# train_loader = DataLoader(train_dataset, batch_size=1024, shuffle=True, num_workers=4)
# val_loader = DataLoader(val_dataset, batch_size=1024, shuffle=False, num_workers=4)

# Training loop with gradient norm monitoring
for batch_idx, (user_features, post_features, labels) in enumerate(tqdm(train_loader)):
    user_features = user_features.to(device)
    post_features = post_features.to(device)
    labels = labels.to(device)
    
    optimizer.zero_grad()
    user_emb, post_emb = model(user_features, post_features)
    scores = torch.sum(user_emb * post_emb, dim=1)
    loss = criterion(scores, labels)
    
    loss.backward()
    
    # Monitor gradients
    total_norm = 0
    for p in model.parameters():
        if p.grad is not None:
            param_norm = p.grad.data.norm(2)
            total_norm += param_norm.item() ** 2
    total_norm = total_norm ** 0.5
    
    optimizer.step()
    
    if batch_idx % 1000 == 0:
        print(f"Batch {batch_idx} statistics:")
        print(f"  Loss: {loss.item():.4f}")
        print(f"  Gradient norm: {total_norm:.4f}")
        print(f"  Score range: [{scores.min().item():.4f}, {scores.max().item():.4f}]")
        print(f"  Prediction mean: {torch.sigmoid(scores).mean().item():.4f}")

  0%|          | 1/26474 [00:02<17:32:25,  2.39s/it]

Batch 0 statistics:
  Loss: 0.7016
  Gradient norm: 0.3347
  Score range: [-0.0336, 0.4606]
  Prediction mean: 0.5540


  0%|          | 105/26474 [00:06<17:52, 24.58it/s] 

Batch 100 statistics:
  Loss: 0.6425
  Gradient norm: 0.2732
  Score range: [-0.9322, 0.9787]
  Prediction mean: 0.5250


  1%|          | 203/26474 [00:10<18:57, 23.09it/s]

...

Batch 6000 statistics:
  Loss: 0.6226
  Gradient norm: 0.1232
  Score range: [-0.9592, 0.9420]
  Prediction mean: 0.5321


 23%|██▎       | 6104/26474 [04:18<11:56, 28.42it/s]

Batch 6100 statistics:
  Loss: 0.6107
  Gradient norm: 0.1418
  Score range: [-0.9666, 0.9800]
  Prediction mean: 0.5400


 23%|██▎       | 6167/26474 [04:20<14:17, 23.67it/s]


KeyboardInterrupt: 